In [8]:
import nltk
import random

In [27]:
# BOT_CONFIG = {
#     "intents": { # Намерения
#         "hello": { # Намерение поздароваться
#             "examples": ["Привет", "Добрый день", "Шалом", "Здравствуйте"],
#             "responses": ["Привет, человек", "Доброго времени суток"]
#         },
#         "bye": {
#             "examples": ["Пока", "Досвидос", "Прощай"],
#             "responses": ["Счастливо", "До свидания", "Если что, возвращайтесь"],
#         },
#         "howdoyoudo": {
#             "examples": ["Как дела", "Что делаешь", "Какие дела"],
#             "responses": ["Маюсь фигней", "Отвечаю на дурацкие вопросы", "Веду вебинары"],
#         },
#     },
#     "failure_phrases": [
#         "Я ничо не понил",
#         "Что-то непонятно",
#         "Я всего лишь бот, сформулируйте попроще"
#     ]
# }

def filter(text):
  text = text.lower()
  text = [c for c in text if c in 'абвгджзеёийклмнопрстуфхцчшщьыъэюя -']
  return ''.join(text)

def match(text, example): # "прощяй!" === "Прощай" ??
  text = filter(text)
  #example = example
  
  distance = nltk.edit_distance(text, example) / len(example)
  if distance < 0.4:
    return True # Текст совпадает
  else:
    return False # Текст НЕ совпадает

In [10]:
def get_intent(text): 
  for intent, data in BOT_CONFIG['intents'].items():
    for example in data['examples']:
      if match(text, example):
        return intent

In [11]:
def get_answer_by_intent(intent):
  phrases = BOT_CONFIG['intents'][intent]['responses']
  return random.choice(phrases)

In [12]:
def bot(text):
  intent = get_intent(text)

  if not intent:
    intent = get_intent_predictive_model(text)

  print("Intent = ", intent)

  if intent:
    return get_answer_by_intent(intent)

  failure_phrases = BOT_CONFIG['failure_phrases']
  return random.choice(failure_phrases)

In [37]:
question = ''
while question not in ['выход', 'отстань']:
  question = input()
  answer = bot(question)
  print(answer)

выход
Intent =  bye
Если что, я тут. Возвращайтесь!


In [14]:
import json

In [16]:
config_file = open("/content/big_bot_config.json", "r")

In [17]:
BOT_CONFIG = json.load(config_file)

In [18]:
len(BOT_CONFIG["intents"])

439

Задача: классифицировать намерения пользователя, т.е. на основе фразы спрогнозировать "интент"

Фразы => Намерение
Х(вход) => Y(выходные)
1. Модель (алгоритм) М.О. = Классификатор (Classifier)
2. Преобразование данных (Векторайзер) Vectorizer
Задача Vectorizer это из текста сделать числа (какие-то и как-то)

In [36]:
X_examples = []
y = []
for intent, data in BOT_CONFIG['intents'].items():
  for example in data['examples']:
    X_examples.append(example)
    y.append(intent)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
count_vecrorizer = CountVectorizer()
count_vecrorizer.fit(X_examples)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [22]:
# list(count_vecrorizer.transform(['ты там че делаешь то']).nonzero()[1])

In [38]:
X = count_vecrorizer.transform(X_examples)

In [39]:
from sklearn.linear_model import LogisticRegression

In [40]:
log_reg = LogisticRegression() #Настройки
log_reg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
log_reg.predict(count_vecrorizer.transform(['ну расскажи анекдот']))

array(['jokes'], dtype='<U29')

In [42]:
log_reg.score(X, y)

0.38840772818121255

In [44]:
from sklearn.svm import LinearSVC

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2,4))
X = tfidf_vectorizer.fit_transform(X_examples)

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.33
)

In [47]:
lin_svc = LinearSVC()
lin_svc.fit(X_train, y_train)
print("Train", lin_svc.score(X_train, y_train))
print("Test", lin_svc.score(X_test, y_test))

Train 0.8632521133764296
Test 0.3249243188698285


In [48]:
def get_intent_predictive_model(text):
  return lin_svc.predict(tfidf_vectorizer.transform([text]))[0]

In [49]:
import pickle

In [52]:
pickle.dump(lin_svc, open("lin_svc.model", "wb"))
pickle.dump(tfidf_vectorizer, open("tfidf_vectorizer.model", "wb"))

In [53]:
! pip install python-telegram-bot --upgrade

Requirement already up-to-date: python-telegram-bot in /usr/local/lib/python3.6/dist-packages (13.1)


In [54]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [55]:
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('I\'m very intelligent AI creature!')


def echo(update: Update, context: CallbackContext) -> None:
    """Echo the user message."""
    answer = bot(update.message.text)
    update.message.reply_text(answer)


def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("1474068577:AAEOuQoOn5p5bTPHeN3NMu74qT7MVmyhq-w", use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on noncommand i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [59]:
main()

Intent =  hello
